## Prompts - Text generally used as instructions to your model
### **Prompt**
What you'll pass to the underlying model

In [1]:
from langchain_community.llms import Ollama
import langchain
langchain.debug = True

llama3_llm = Ollama(model="llama3")
prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

print(llama3_llm.invoke(prompt))

[llm/start] [llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "Today is Monday, tomorrow is Wednesday.\n\nWhat is wrong with that statement?"
  ]
}
[llm/end] [llm:Ollama] [1.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "A classic lateral thinking puzzle!\n\nThe answer is: \"Tomorrow can't be Wednesday if today is Monday.\"\n\nIn our normal understanding of the calendar, today being a Monday means that tomorrow would be Tuesday, not Wednesday. So, there's a logical inconsistency in the statement. Well done on crafting a clever puzzle!",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-07-15T13:23:38.289429Z",
          "response": "",
          "done": true,
          "done_reason": "stop",
          "context": [
            128006,
            882,
            128007,
            1432,
            15724,
            374,
            7159,
            11,
            16986,
            374,
     

### **Prompt Template**
An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

Think of it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts

*Advanced: Check out LangSmithHub(https://smith.langchain.com/hub) for many more community prompt templates*

In [2]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

llm = Ollama(model="llama3")

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm.invoke(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
[llm/start] [llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "I really want to travel to Rome. What should I do there?\n\nRespond in one short sentence"
  ]
}
[llm/end] [llm:Ollama] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Explore the ancient ruins of the Colosseum, Roman Forum, and Pantheon, and indulge in delicious Italian cuisine and gelato throughout your stay.",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-07-15T13:25:44.974428Z",
          "response": "",
          "done": true,
          "done_reason": "stop",
          "context": [
            128006,
            882,
            128007,
            1432,
            40,
            2216,
            1390,
            311,
            5944,
            311,
            22463,
            13,
            3639,
      

### **Example Selectors**
An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

Check out different types of example selectors [here](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/)

If you want an overview on why examples are important (prompt engineering), check out [this video](https://www.youtube.com/watch?v=dOxUroR57xs)

In [4]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [5]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OllamaEmbeddings(model='llama3'), 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=2
)

In [6]:
# FewShotPromptTemplate -
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

In [9]:
# Select a noun!
# my_noun = "plant"
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: tree
Example Output: ground

Input: student
Output:


In [11]:
llm.invoke(similar_prompt.format(noun=my_noun))

[llm/start] [llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "Give the location an item is usually found in\n\nExample Input: driver\nExample Output: car\n\nExample Input: tree\nExample Output: ground\n\nInput: student\nOutput:"
  ]
}
[llm/end] [llm:Ollama] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I think I can help with that!\n\nThe answer would be \"desk\".",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-07-15T13:36:14.498904Z",
          "response": "",
          "done": true,
          "done_reason": "stop",
          "context": [
            128006,
            882,
            128007,
            271,
            36227,
            279,
            3813,
            459,
            1537,
            374,
            6118,
            1766,
            304,
            271,
            13617,
            5688,
            25,
            5696,
            198,
            13617,
   

'I think I can help with that!\n\nThe answer would be "desk".'

### **Output Parsers Method 1: Prompt Instructions & String Parsing**
A helpful way to format the output of a model. Usually used for structured output. LangChain has a bunch more output parsers listed on their [documentation](https://python.langchain.com/docs/modules/model_io/output_parsers).

Two big concepts:

**1. Format Instructions** - A autogenerated prompt that tells the LLM how to format it's response based off your desired result

**2. Parser** - A method which will extract your model's text output into a desired structure (usually json)

In [12]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain import PromptTemplate
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [13]:
# How you would like your response structured. This is basically a fancy prompt template
llm = Ollama(model="llama3")

response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [14]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [15]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



In [16]:
llm_output = llm.invoke(promptValue)
llm_output

[llm/start] [llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "You will be given a poorly formatted string from a user.\nReformat it and make sure all the words are spelled correctly\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"bad_string\": string  // This a poorly formatted user input string\n\t\"good_string\": string  // This is your response, a reformatted response\n}\n```\n\n% USER INPUT:\nwelcom to califonya!\n\nYOUR RESPONSE:"
  ]
}
[llm/end] [llm:Ollama] [2.05s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here's the response:\n\n```json\n{\n    \"bad_string\": \"welcom to califonya!\",\n    \"good_string\": \"Welcome to California!\"\n}\n```",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-07-15T13:43:10.116898Z",
          "response": "",
          "done": true,
          

'Here\'s the response:\n\n```json\n{\n    "bad_string": "welcom to califonya!",\n    "good_string": "Welcome to California!"\n}\n```'

In [17]:
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

### **Output Parsers Method 2: OpenAI Fuctions**
When OpenAI released function calling, the game changed. This is recommended method when starting out.

They trained models specifically for outputing structured data. It became super easy to specify a Pydantic schema and get a structured output.

There are many ways to define your schema, I prefer using Pydantic Models because of how organized they are. Feel free to reference OpenAI's [documention](https://platform.openai.com/docs/guides/gpt/function-calling) for other methods.

In order to use this method you'll need to use a model that supports [function calling](https://openai.com/blog/function-calling-and-other-api-updates#:~:text=Developers%20can%20now%20describe%20functions%20to%20gpt%2D4%2D0613%20and%20gpt%2D3.5%2Dturbo%2D0613%2C). I'll use `gpt4-0613`

**Example 1: Simple**

Let's get started by defining a simple model for us to extract from.

In [19]:
import os
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
from typing import Optional
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]
class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [24]:
from langchain.chains.openai_functions import create_structured_output_chain

llm = ChatOpenAI(model='gpt-4-0613', openai_api_key=openai_api_key)

chain = create_structured_output_chain(Person, llm, prompt)
print(prompt)
chain.run(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

input_variables=['user_input'] partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"bad_string": string  // This a poorly formatted user input string\n\t"good_string": string  // This is your response, a reformatted response\n}\n```'} template='\nYou will be given a poorly formatted string from a user.\nReformat it and make sure all the words are spelled correctly\n\n{format_instructions}\n\n% USER INPUT:\n{user_input}\n\nYOUR RESPONSE:\n'
[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "user_input": "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
}
[llm/start] [chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou will be given a poorly formatted string from a user.\nReformat it and make sure all the words are spelled correctly\n\nThe outp

Person(name='Sally, Joey, Caroline', age=13, fav_food='spinach')

Notice how we only have data on one person from that list? That is because we didn't specify we wanted multiple. Let's change our schema to specify that we want a list of people if possible.


In [ ]:
from typing import Sequence

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")

In [ ]:
chain = create_structured_output_chain(People, llm, prompt)
chain.run(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

# Workshop exercise:

**Exercise 1:**
1. Create a schema for a `Car` with the following fields:
    - make: str
    - model: str
    - year: int
    - color: str
    - price: float
2. create a prompt to extract information about a car from a string
3. Run the prompt and parse the output
4. Print the output